## CIFAR 10

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

You can get the data via:

    wget http://pjreddie.com/media/files/cifar.tgz

In [2]:
from fastai.conv_learner import *
PATH = "data/cifar10/"
os.makedirs(PATH,exist_ok=True)

In [3]:
classes = ('plane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

In [4]:
def get_data(sz,bs, partial_train_classes=None, max_train_per_class=None):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, partial_train_classes=partial_train_classes, max_train_per_class=max_train_per_class)

In [5]:
bs=256

## Fully connected model

In [6]:
data = get_data(32,bs)

In [14]:
lr=.5e-2

From [this notebook](https://github.com/KeremTurgutlu/deeplearning/blob/master/Exploring%20Optimizers.ipynb) by our student Kerem Turgutlu:

## Refactored

In [8]:
class ConvLayer(nn.Module):
    def __init__(self, ni, nf):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, kernel_size=3, stride=2, padding=1)
        
    def forward(self, x): return F.relu(self.conv(x))

## BatchNorm

In [9]:
class BnLayer(nn.Module):
    def __init__(self, ni, nf, stride=2, kernel_size=3):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, kernel_size=kernel_size, stride=stride,
                              bias=False, padding=1)
        self.a = nn.Parameter(torch.zeros(nf,1,1))
        self.m = nn.Parameter(torch.ones(nf,1,1))
        
    def forward(self, x):
        x = F.relu(self.conv(x))
        x_chan = x.transpose(0,1).contiguous().view(x.size(1), -1)
        if self.training:
            self.means = x_chan.mean(1)[:,None,None]
            self.stds  = x_chan.std (1)[:,None,None]
        return (x-self.means) / self.stds *self.m + self.a

## Resnet

In [10]:
class ResnetLayer(BnLayer):
    def forward(self, x): return x + super().forward(x)

## Resnet 2

In [11]:
class Resnet2(nn.Module):
    def __init__(self, layers, c, p=0.5):
        super().__init__()
        self.conv1 = BnLayer(3, 16, stride=1, kernel_size=7)
        self.layers = nn.ModuleList([BnLayer(layers[i], layers[i+1])
            for i in range(len(layers) - 1)])
        self.layers2 = nn.ModuleList([ResnetLayer(layers[i+1], layers[i + 1], 1)
            for i in range(len(layers) - 1)])
        self.layers3 = nn.ModuleList([ResnetLayer(layers[i+1], layers[i + 1], 1)
            for i in range(len(layers) - 1)])
        self.out = nn.Linear(layers[-1], c)
        self.drop = nn.Dropout(p)
        
    def forward(self, x):
        x = self.conv1(x)
        for l,l2,l3 in zip(self.layers, self.layers2, self.layers3):
            x = l3(l2(l(x)))
        x = F.adaptive_max_pool2d(x, 1)
        x = x.view(x.size(0), -1)
        x = self.drop(x)
        return F.log_softmax(self.out(x), dim=-1)

## Baseline

In [16]:
learn = ConvLearner.from_model_data(Resnet2([16, 32, 64, 128, 256], 10, 0.2), get_data(32,bs))
%time learn.fit(lr, 20)
%time learn.fit(lr/3, 10)

HBox(children=(IntProgress(value=0, description='Epoch'), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.62083    1.445231   0.4779    
    1      1.41125    1.374444   0.5281                     
    2      1.271459   1.189134   0.5743                     
    3      1.170695   1.163856   0.593                      
    4      1.088109   1.019313   0.6384                     
    5      1.013146   0.977052   0.6523                     
    6      0.94332    0.881877   0.692                       
    7      0.875162   0.818677   0.7112                      
    8      0.830351   0.834888   0.7084                      
    9      0.785486   0.778031   0.7282                      
    10     0.76246    0.739769   0.7438                      
    11     0.727989   0.703775   0.7541                      
    12     0.68883    0.699439   0.7581                      
    13     0.659834   0.656081   0.7727                      
    14     0.641092   0.655555   0.7751                      
    15     0.627623   0.68061   

KeyboardInterrupt: 

Converged after 36 epochs to 83.22% accuracy


## Jeremy

In [23]:
learn = ConvLearner.from_model_data(Resnet2([16, 32, 64, 128, 256], 10, 0.2), get_data(32,bs))
learn.set_data(get_data(32,bs,partial_train_classes=range(2)))
learn.unfreeze()
learn.fit(lr, 4)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                  
    0      0.468652   30.477898  0.1789    
    1      0.307907   31.855173  0.1802                    
    2      0.248821   36.604384  0.1843                    
    3      0.219033   34.605785  0.1851                    



[array([34.60579]), 0.1851]

In [24]:
learn.save('resnet_jeremy_0_2')

In [25]:
learn.load('resnet_jeremy_0_2')

In [26]:
learn.set_data(get_data(32,bs,partial_train_classes=range(4)))
learn.unfreeze()
%time learn.fit(lr, 5)

HBox(children=(IntProgress(value=0, description='Epoch', max=5), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                 
    0      1.3251     8.458939   0.2489    
    1      0.950242   9.966445   0.2654                    
    2      0.831157   8.31221    0.2687                    
    3      0.779131   7.232314   0.2784                    
    4      0.735974   9.542857   0.2801                    

CPU times: user 38.2 s, sys: 17.7 s, total: 55.9 s
Wall time: 30.1 s


[array([9.54286]), 0.2801]

In [27]:
learn.save('resnet_jeremy_0_4')

In [32]:
learn.load('resnet_jeremy_0_4')

In [33]:
learn.set_data(get_data(32,bs,partial_train_classes=range(6)))
learn.unfreeze()
%time learn.fit(lr, 10)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.204459   8.790785   0.3366    
    1      1.094665   8.048182   0.353                      
    2      1.035795   8.919142   0.3755                     
    3      0.998177   8.241338   0.3789                      
    4      0.948118   9.614997   0.3777                      
    5      0.920844   9.02059    0.4011                      
    6      0.890956   25.686805  0.3824                      
    7      0.848544   6.956179   0.4062                      
    8      0.820532   10.665026  0.4206                      
    9      0.792623   5.48288    0.4188                      

CPU times: user 1min 43s, sys: 48.6 s, total: 2min 32s
Wall time: 1min 23s


[array([5.48288]), 0.4188]

In [34]:
learn.save('resnet_jeremy_0_6')

In [35]:
learn.load('resnet_jeremy_0_6')

In [36]:
learn.set_data(get_data(32,bs,partial_train_classes=range(8)))
learn.unfreeze()
%time learn.fit(lr, 10)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.989394   4.503135   0.5343    
    1      0.897312   3.540402   0.5575                      
    2      0.850951   3.15485    0.5743                      
    3      0.819025   2.861161   0.5782                      
    4      0.776574   5.463484   0.5628                      
    5      0.733169   2.882437   0.5873                      
    6      0.716626   3.54237    0.5972                      
    7      0.695866   6.397343   0.5882                      
    8      0.672386   2.749968   0.6018                      
    9      0.653743   9.348982   0.5732                      

CPU times: user 2min 12s, sys: 59.8 s, total: 3min 12s
Wall time: 1min 46s


[array([9.34898]), 0.5732]

In [37]:
learn.save('resnet_jeremy_0_8')

In [40]:
learn.load('resnet_jeremy_0_8')

In [41]:
learn.set_data(get_data(32,bs))
learn.unfreeze()
%time learn.fit(lr, 10)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.874271   0.819114   0.7182    
    1      0.758142   0.766285   0.7384                      
    2      0.709677   0.725749   0.7482                      
    3      0.661929   0.703542   0.7582                      
    4      0.625835   0.658224   0.7772                      
    5      0.622454   0.65262    0.7765                      
    6      0.601451   0.637579   0.78                        
    7      0.572113   0.621003   0.7869                      
    8      0.54701    0.666955   0.7765                      
    9      0.534824   0.57187    0.8016                      

CPU times: user 2min 42s, sys: 1min 12s, total: 3min 54s
Wall time: 2min 10s


[array([0.57187]), 0.8016]

In [42]:
learn.save('resnet_jeremy_1_10')

In [52]:
learn.load('resnet_jeremy_1_10')

In [53]:
learn.set_data(get_data(32,bs))
learn.unfreeze()
%time learn.fit(lr/3, 5)

HBox(children=(IntProgress(value=0, description='Epoch', max=5), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.465095   0.572049   0.8083    
    1      0.449578   0.54239    0.8155                      
    2      0.437213   0.549003   0.8117                      
    3      0.434965   0.532798   0.8202                      
    4      0.419015   0.552882   0.8087                      

CPU times: user 1min 20s, sys: 36.7 s, total: 1min 56s
Wall time: 1min 5s


[array([0.55288]), 0.8087]

In [54]:
learn.save('resnet_jeremy_1')

After seeing 6 times the number of images in the dataset ((0.2+0.4+0.6+0.8+1)*2=6), the network obtained an accuracy of 0.58

## Leslie

In [55]:
n_trn = data.trn_ds.n
n_classes = data.trn_ds.c

In [146]:
learn = ConvLearner.from_model_data(Resnet2([16, 32, 64, 128, 256], 10, 0.2), get_data(32,bs))
learn.set_data(get_data(32,bs,max_train_per_class=int(0.1*(n_trn/n_classes))))
learn.unfreeze()
learn.fit(lr/3, 10)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                 
    0      2.651345   2.144208   0.2592    
    1      2.327097   1.861337   0.3359                   
    2      2.119895   1.726544   0.3735                   
    3      1.9917     1.655122   0.3974                   
    4      1.887938   1.624999   0.4078                   
    5      1.804281   1.581374   0.4148                   
    6      1.726627   1.550487   0.4355                   
    7      1.665821   1.523349   0.4425                   
    8      1.61548    1.537576   0.4367                   
    9      1.570571   1.490576   0.4556                   



[array([1.49058]), 0.4556]

In [147]:
learn.save('resnet_leslie_0_1')

In [148]:
learn.load('resnet_leslie_0_1')

In [149]:
learn.set_data(get_data(32,bs,max_train_per_class=int(0.4*(n_trn/n_classes))))
learn.unfreeze()
learn.fit(lr/2, 10)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                 
    0      1.57497    1.456482   0.4674    
    1      1.498303   1.423247   0.4854                   
    2      1.448542   1.443717   0.4867                   
    3      1.375175   1.291227   0.5383                   
    4      1.318444   1.291794   0.5344                   
    5      1.270394   1.262013   0.5521                   
    6      1.234296   1.270916   0.547                    
    7      1.183839   1.229341   0.5765                   
    8      1.137935   1.152714   0.5873                   
    9      1.113853   1.108869   0.6061                   



[array([1.10887]), 0.6061]

In [150]:
learn.save('resnet_leslie_0_4')

In [151]:
learn.load('resnet_leslie_0_4')

In [152]:
learn.set_data(get_data(32,bs,max_train_per_class=int(0.6*(n_trn/n_classes))))
learn.unfreeze()
learn.fit(lr, 10)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.20431    1.273722   0.5692    
    1      1.120214   1.19144    0.5963                     
    2      1.05785    1.007561   0.6426                     
    3      0.992114   0.96356    0.6575                      
    4      0.936213   1.014417   0.6469                      
    5      0.891578   0.92145    0.6797                      
    6      0.861544   1.02679    0.6616                      
    7      0.826873   0.831323   0.7121                      
    8      0.803495   0.857969   0.7061                      
    9      0.763209   0.873475   0.6992                      



[array([0.87348]), 0.6992]

In [153]:
learn.save('resnet_leslie_0_6')

In [154]:
learn.load('resnet_leslie_0_6')

In [155]:
learn.set_data(get_data(32,bs,max_train_per_class=int(0.8*(n_trn/n_classes))))
learn.unfreeze()
learn.fit(lr, 4)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.781522   0.797074   0.7255    
    1      0.751392   0.754069   0.7388                      
    2      0.706199   0.70011    0.7559                      
    3      0.688671   0.736564   0.7442                      



[array([0.73656]), 0.7442]

In [156]:
learn.save('resnet_leslie_0_8')

In [161]:
learn.load('resnet_leslie_0_8')

In [162]:
learn.set_data(get_data(32,bs))
learn.unfreeze()
learn.fit(lr/2, 10)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.626139   0.658674   0.7725    
    1      0.590702   0.671947   0.7696                      
    2      0.589575   0.62845    0.7826                      
    3      0.570031   0.614408   0.7896                      
    4      0.561938   0.623096   0.7895                      
    5      0.536144   0.61554    0.7924                      
    6      0.529773   0.623704   0.7856                      
    7      0.510894   0.587844   0.8027                      
    8      0.494644   0.597669   0.7975                      
    9      0.491722   0.636317   0.7924                      



[array([0.63632]), 0.7924]

In [163]:
learn.save('resnet_leslie_1')

In [164]:
learn.load('resnet_leslie_1')

## Verdict

1st place - Baseline - 67%<br/>
2nd place - Leslie - 66.6%<br/>
3rd place - Jeremy - 58%<br/>
